# Solving the 15 Puzzle using Dynamic Programming

The 15-puzzle is basically a 4x4 square with 15 movable tiles in it. There is a single tile vacancy in the box, which can be used to shift tiles around.
The final solved puzzle looks like this:
<table>
  <tr>
    <td>1</td>
    <td>2</td>
    <td>3</td>
    <td>4</td>
  </tr>
  <tr>
    <td>5</td>
    <td>6</td>
    <td>7</td>
    <td>8</td>
  </tr>
  <tr>
    <td>9</td>
    <td>10</td>
    <td>11</td>
    <td>12</td>
  </tr>
  <tr>
    <td>13</td>
    <td>14</td>
    <td>15</td>
    <td></td>
  </tr>
</table>

It is important to note that not every permutation can reach the goal, hence we must generate possible permutations from the goal state itself.

If you play this puzzle, you would naturally think that it is probably a good idea to start fixing the first row(1, 2, 3, 4) and then the second row(5, 6, 7, 8) and then the third and fourth rows. While you are trying to put 1, 2, 3 and 4 into their places, you probably don’t care about the other numbers. They all would look the same to you as if they don’t exist and their number doesn’t matter. And that is the main idea we use here to reduce the total number of states.

In [4]:
import numpy as np
import random
from more_itertools import distinct_permutations
import time

In [5]:
class fifteenstate:
    global step
    step=0
    def __init__(self):
        self.gamma=0.9
        self.num_cells=16
        #Masking solved numbers as -1 and remaining as 0
        self.first_solved = [1, 2, 3, 4, self.num_cells, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.second_solved= [-1, -1, -1, -1, 5, 6, 7, 8, self.num_cells, 0, 0, 0, 0, 0, 0, 0]
        self.all_solved= [-1, -1, -1, -1, -1, -1, -1, -1, 9, 10, 11, 12, 13, 14, 15, self.num_cells]
        
        self.phase1_states = self.generate_states([1,2,3,4,16], 0, 11)
        self.phase1_policy = self.pol_iter(self.phase1_states)

        self.phase2_states = self.generate_states([5,6,7,8,16], 4, 7)
        self.phase2_policy = self.pol_iter(self.phase2_states)

        self.phase3_states = self.generate_states([9,10,11,12,13,14,15,16], 8, 0)
        self.phase3_policy = self.pol_iter(self.phase3_states)

    def possible_actions(self, state):
        actions=[]
        i=state.index(self.num_cells)
        row=i//4
        col=i%4
        if row > 0:
            actions.append("Up")
        if row < 3:
            actions.append("Down")
        if col > 0:
            actions.append("Left")
        if col < 3:
            actions.append("Right")
        return actions
    
    def move(self, state, action):
        i=state.index(self.num_cells)
        row=i//4
        col=i%4
        if action == "Up" and row > 0 and state[i - 4] != -1:
            state[i], state[i - 4] = state[i - 4], state[i]
        elif action == "Down" and row < 3 and state[i + 4] != -1:
            state[i], state[i + 4] = state[i + 4], state[i]
        elif action == "Left" and col > 0 and state[i - 1] != -1:
            state[i], state[i - 1] = state[i - 1], state[i]
        elif action == "Right" and col < 3 and state[i + 1] != -1:
            state[i], state[i + 1] = state[i + 1], state[i]
        return state
    
    def partial_solved(self, state):
        return (state[:4] == self.first_solved[:4] or state[:8] == self.second_solved[:8] or state == self.all_solved)
    
    def reward(self, state):
        if(self.partial_solved(state)):
            return 1000
        else:
            return 0
    
    def mask(self, state, phase):
        if phase == 1:
            return [x if x in [1, 2, 3, 4, 16] else 0 for x in state]
        elif phase == 2:
            return [x if x in [5, 6, 7, 8, 16] else (-1 if x in [1, 2, 3, 4] else 0) for x in state]
        elif phase == 3:
            return [x if x in [9, 10, 11, 12, 13, 14, 15, 16] else -1 for x in state]
        return state

    def pol_improv(self, states, pol, val):
        stable = True
        for state in states:
            state_tuple = tuple(state)
            prev_action = pol[state_tuple] 

            if self.partial_solved(state):
                continue

            max_value = -np.inf  
            best_action = ''
            
            for action in self.possible_actions(state):
                new_state = state.copy()
                new_state = self.move(new_state, action)
                action_value = self.reward(new_state) + self.gamma * val[tuple(new_state)]
                if action_value > max_value:
                    max_value = action_value
                    best_action = action
            
            pol[state_tuple] = best_action  #Update policy for a given value function
            if best_action != prev_action:
                stable = False
        return stable
    
    def pol_eval(self, states, pol, val):
        for state in states:
            state_tuple = tuple(state)  
            if self.partial_solved(state):  #Every policy will contain actions only till its partial-goal state, hence skip goal state
                continue

            action = pol[state_tuple]
            new_state=state.copy()
            new_state = self.move(new_state, action)  
            val[state_tuple] = self.reward(new_state) + self.gamma*val[tuple(new_state)] #For a given policy update the value function

    def pol_iter(self, states):
        val = {tuple(state): 0 for state in states}  
        pol = {tuple(state): '' for state in states} 

        stable = False
        while not stable:
            stable = self.pol_improv(states, pol, val)  
            self.pol_eval(states, pol, val)  
        return pol
    
    def generate_states(self, distinct_numbers, num_minus_ones, num_zeros):
        arr = distinct_numbers + [0]*num_zeros
        result=[]
        for perm in distinct_permutations(arr):
            result.append([-1] * num_minus_ones + list(perm))  
        return result
    
    def scramble(self, num_steps=300):
        solved_puzzle = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]  
        current_state = solved_puzzle.copy()
        for _ in range(num_steps):
            act=self.possible_actions(current_state)
            action = random.choice(act)  # Choose a random move
            current_state = self.move(current_state, action)  # Apply the move
        return current_state
    
    def display(self, state):
        global step
        step += 1
        for i in range(0, 16, 4):
            row = ["|"] 
            for num in state[i:i+4]:  
                row.append(f" {num:2d} ")  
            row.append("|") 
            print("".join(row))  
        print(f"Step: {step}")  
        print("----------------------------")

    def solve(self, puzzle):
        state = puzzle.copy()

        def process_phase(masked_state, policy):
            actions = []
            while True:
                if tuple(masked_state) in policy:
                    action = policy[tuple(masked_state)]
                    if action not in ["Up", "Down", "Left", "Right"]:
                        break
                    actions.append(action)
                    masked_state = self.move(masked_state, action)
                else:
                    break
            return actions

        masked_state = self.mask(state, 1)
        actions_phase1 = process_phase(masked_state, self.phase1_policy)

        for action in actions_phase1:
            state = self.move(state, action)
            self.display(state)
            time.sleep(1)

        masked_state = self.mask(state, 2)
        actions_phase2 = process_phase(masked_state, self.phase2_policy)

        for action in actions_phase2:
            state = self.move(state, action)
            self.display(state)
            time.sleep(1)

        masked_state = self.mask(state, 3)
        actions_phase3 = process_phase(masked_state, self.phase3_policy)

        for action in actions_phase3:
            state = self.move(state, action)
            self.display(state)
            time.sleep(1)

        print("SOLVED!!!")

In [6]:
puzzle=fifteenstate()
start=puzzle.scramble()
print("Let us solve the folowing:")
puzzle.display(start)
puzzle.solve(start)

Let us solve the folowing:
|  7   8   6   4 |
|  9   5   2   1 |
|  3  10  15  14 |
| 13  12  11  16 |
Step: 1
----------------------------
|  7   8   6   4 |
|  9   5   2   1 |
|  3  10  15  16 |
| 13  12  11  14 |
Step: 2
----------------------------
|  7   8   6   4 |
|  9   5   2   1 |
|  3  10  16  15 |
| 13  12  11  14 |
Step: 3
----------------------------
|  7   8   6   4 |
|  9   5   2   1 |
|  3  16  10  15 |
| 13  12  11  14 |
Step: 4
----------------------------
|  7   8   6   4 |
|  9  16   2   1 |
|  3   5  10  15 |
| 13  12  11  14 |
Step: 5
----------------------------
|  7   8   6   4 |
|  9   2  16   1 |
|  3   5  10  15 |
| 13  12  11  14 |
Step: 6
----------------------------
|  7   8   6   4 |
|  9   2   1  16 |
|  3   5  10  15 |
| 13  12  11  14 |
Step: 7
----------------------------
|  7   8   6   4 |
|  9   2   1  15 |
|  3   5  10  16 |
| 13  12  11  14 |
Step: 8
----------------------------
|  7   8   6   4 |
|  9   2   1  15 |
|  3   5  16  10 |
| 13  12  11